# Семестровый проект.
# Рекомендательная система для сериалов

#### Шадрин Сергей, Слепцов Александр, Данько Артем

### Задачи и цели

Цель проекта — на примере сайта myshows.me создать рекомендательную систему для сериалов, то есть уметь предсказывать рейтинг непросмотренного сериала для какого либо пользователя.

Поставленные задачи:
- Сбор и подготовка данных;
- Выбор модели и метрики для оценки качества предсказания;
- Применение на практике.

### Сбор данных

Для начала с сайта был собран список из 150 тысяч пользователей. На странице пользователя отображаются его оценки, выставленные сериалам, причем сериалы делятся на следующие категории:
  - "смотрит" (просматриваемые в данный момент),
  - "будет смотреть" (сюда также включены еще невышедшие сериалы),
  - "перестал" (сериалы, просмотр которых прекращен),
  - "полностью посмотрел" (соответственно, просмотренные). 

Далее для каждого из 150 тысяч пользователей был получен список отмеченных им сериалов. Получилось 13096257 записей. Полученные данные состоят из трех столбцов: порядковый номер обработанной ссылки пользователя, ID сериала на сайте, выставленная этому сериалу оценка. 

Значения последнего столбца находится в диапазоне от 0 до 5, где 0 означает, что оценка данному сериалу не была выставлена. Данные с невыставленной оценкой не несут особой ценности, поэтому убираем их. В итоге мы получили 7112074 записей.

Страница с пользователями myshows.me/community/users/

<img src="rec_sys_images/users_page.png">

Страница пользователя myshows.me/username

<img src="rec_sys_images/user_page.png">

Функции для парсинга страниц:

```python
site = 'https://myshows.me/community/users/?page={}'
max_num_page = 5000

def extract_users_from_page(site_num):
    soup = BeautifulSoup(requests.get(site.format(site_num)).text, 'html.parser')
    res = []
    
    links = soup.find_all('a', class_='userBlock linkBlock')
    for link in links:
        res.append(link.attrs['href'])
    
    return res

def extract_grades_from_user_page(user_url):
    soup = BeautifulSoup(requests.get(user_url).text, 'html.parser')
    res = {}
    
    tabs = soup.find_all('div', class_='tabs_cont')
    
    for completed_tab in tabs:
        all_grades = completed_tab.find_all('tr')
        for grade in all_grades[:-1]:
            grade_parts = grade.find_all('td')
            id_s = grade_parts[0].find('a').attrs['href'].split('/')[-2]
            grade_value = grade_parts[1].find('span').attrs['class'][1][1]
            res[id_s] = grade_value
    
    return res
```

In [80]:
%matplotlib inline

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook

from scipy.sparse.linalg import svds
from scipy.sparse import coo_matrix
from sklearn.preprocessing import LabelEncoder

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12, 6)

from scipy.spatial.distance import cosine
from scipy.spatial.distance import pdist
from scipy.spatial.distance import squareform
from scipy.spatial.distance import correlation
from scipy.stats import pearsonr
from sklearn.metrics import pairwise_distances
import math

from sklearn.metrics import mean_squared_error
from sklearn.neighbors import NearestNeighbors

import warnings
warnings.filterwarnings("ignore")

In [3]:
df_ratings = pd.read_csv('../data/ratings_without_zeros.csv',
                         header=None,
                         names=['user_id', 'show_id', 'rating_val'],
                         sep=' ')

df_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7112074 entries, 0 to 7112073
Data columns (total 3 columns):
user_id       int64
show_id       int64
rating_val    int64
dtypes: int64(3)
memory usage: 162.8 MB


In [4]:
print(df_ratings.shape)
df_ratings.head(10)

(7112074, 3)


,user_id,show_id,rating_val
0,1,55877,4
1,1,55672,4
2,1,41907,4
3,1,34737,5
4,1,44997,5
5,1,50726,4
6,1,48017,4
7,1,42707,3
8,1,26428,5
9,1,331,4


### Подготовка данных

Разобьем данные на обучение и контроль в пропорции 95/5.

В полном объеме датасет не будем использовать. Возьмем первые 100000 строк, из них на обучение — 95000, на контроль — 5000. Для этого каждую 20-ую строку из 100000 будем брать в контроль.

В качестве метрики для качества будет использовать MSE:
    $$ MSE = \sum_{(u, m)}(r_{um} - y_{um})^{2} $$

Здесь $r_{um}$ — предсказанный рейтинг фильма $m$ для пользователя $u$, $y_{um}$ — рейтинг, который в действительности проставил пользователь.

In [7]:
train, test = df_ratings.iloc[:100000], df_ratings.iloc[range(0, 100000, 20)]
train = train.drop(range(0, 100000, 20), axis=0)

In [10]:
# Преобразуем train в матрицу
train_table = train.pivot(index='user_id', columns='show_id', values='rating_val')
print(train_table.shape)
train_table.head()

(743, 8860)


show_id,1,2,3,4,5,6,7,8,9,10,...,59296,59297,59307,59314,59331,59347,59352,59364,59411,59414
user_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,5.0,NaN,NaN,NaN,5.0,5.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5.0,5.0,3.0,NaN,NaN,3.0,3.0,4.0,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5.0,NaN,5.0,NaN,NaN,NaN,5.0,5.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Random алгоритм

Посмотреть результат если будем предсказывать случайным образом.

In [311]:
pred = np.random.randint(1, 5, len(test))

print(mean_squared_error(test.values[:, 2], pred))

4.1628


### Baseline алгоритм

С помощью mse этого алгоритма будем оценивать последующие модели.
Будем считать оценку пользователя $u$ для товара $i$ как $b_{ui}$:

* $b_{ui} = \mu + b_u + b_i$,
* $b_{u} = \frac{1}{|I_u|+\alpha}\sum_{i\in I_u}(R_{ui} - \mu)$
* $b_{i} = \frac{1}{|U_i|+\beta}\sum_{u\in U_i}(R_{ui} - b_u - \mu)$

Интерпретация:
* $b_u$ — насколько выше (ниже) среднего пользователь оценивает товары;
* $b_i$ — насколько выше (ниже) среднего оценивается товар;
* $\mu$ — просто общий средний рейтинг;
* $U_i$ — множество пользователей, оценивших товар $i$;
* $I_u$ — множество товаров, оценненных пользователем $u$;
* $\alpha$, $\beta$ — коэффиценты для сглаживания.

In [125]:
matr = train_table.values

mu = matr[~np.isnan(matr)].mean()
alpha = 0.1
beta = 0.1

pred = np.zeros(len(test))

for i in range(len(pred)):
    uid, sid, _ = list(test.values[i, :])

    ind1 = (train_table.index.values == uid)
    ind2 = (train_table.columns.values == sid)
    if len(ind1[ind1 == True]) == 0 or \
        len(ind2[ind2 == True]) == 0:
        pred[i] = 0
        continue
    
    b_u = 0
    row = matr[ind1, :]
    row_len = len(row[~np.isnan(row)])
    row_sum = row[~np.isnan(row)].sum()
    b_u = (row_sum - row_len * mu) / (row_len + alpha)
    
    b_i = 0
    col = matr[:, ind2]
    col_len = len(col[~np.isnan(col)])
    col_sum = col[~np.isnan(col)].sum()
    b_i = (col_sum - col_len * (mu + b_u)) / (col_len + beta)
    
    pred[i] = mu + b_u + b_i

print(mean_squared_error(test.values[:, 2], pred))

1.5696947424394272


### User-based CF

Так как количество юзеров у нас меньше, чем объектов, будем использовать User-based алгоритм:

* Посчитаем сходство между пользователями $s \in \mathbb{R}^{U \times U}$
* Для целевого пользователя $u$ найти похожих пользователей $N(u)$
$$ \hat{R}_{ui} = \bar{R}_u + \frac{\sum_{v \in N(u)} s_{uv}(R_{vi} - \bar{R}_v)}{\sum_{v \in N(u)} \left| s_{uv}\right|} $$

* $\bar{R}_u$ - поправка на писсимизм\оптимизм пользователей

В качестве функции сходства будем использовать корреляцию Пирсона. Похожих пользователей будем рассматривать, только если в пересечении сериалов, которые они посмотрели, больше 10 элементов. Для получения оценки будем брать
100 ближайших юзеров.

In [115]:
def my_similarity(u, v):
    ind = ((u != 0) & (v != 0))
    
    if len(u[ind]) <= 10:
        return 0
    
    return abs(pearsonr(u[ind], v[ind])[0])

In [116]:
# для pairwise_distances требуется, чтобы не было NaN'ов
# поэтому заменяем NaN на 0
res_corr = pairwise_distances(train_table.fillna(0).values, metric=my_similarity)
np.fill_diagonal(res_corr, 0)
res_corr = np.abs(res_corr)
res_corr

array([[0.        , 0.        , 0.49922104, ..., 0.        , 0.43222147,
        0.33587572],
       [0.        , 0.        , 0.05827165, ..., 0.        , 0.12309149,
        0.09090909],
       [0.49922104, 0.05827165, 0.        , ..., 0.        , 0.39700976,
        0.22402048],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.43222147, 0.12309149, 0.39700976, ..., 0.        , 0.        ,
        0.18058695],
       [0.33587572, 0.09090909, 0.22402048, ..., 0.        , 0.18058695,
        0.        ]])

In [312]:
matr = train_table.fillna(0).values

pred = np.zeros(len(test))

for i in range(len(pred)):
    uid, sid, _ = list(test.values[i, :])

    ind1 = (train_table.index.values == uid)
    ind2 = (train_table.columns.values == sid)

    if len(ind1[ind1 == True]) == 0 or \
        len(ind2[ind2 == True]) == 0:
        pred[i] = 0
        continue
        
    inds_knear = np.argpartition(-res_corr[ind1, :].flatten(), 100)[:100]    
    u_inds_knear = []
    
    for j in inds_knear:
        row = matr[j, :]
        if len(row[ind2]) != 0 and not math.isnan(row[ind2]):
            u_inds_knear.append(j)
    
    if len(u_inds_knear) == 0:
        pred[i] = 0
        continue
    
    s = 0
    z = 0
    
    for j in inds_knear:
        row = matr[j, :]
        s += (row[ind2] - row[row != 0].mean()) * res_corr[ind1, j]
        z += abs(res_corr[ind1, j])
    
    row = matr[ind1, :]
    t = row[row != 0].mean() + s / z
        
    if math.isnan(t) or t < 0:
        t = 0
        
    pred[i] = t

print(mean_squared_error(test.values[:, 2], pred))

11.605216478293094


### Модели со скрытыми факторами. SVD

Для каждого пользователя и товара построим векторы $p_u\in \mathbb{R}^{k}$ и $q_i \in \mathbb{R}^{k}$ так, чтобы
$$ R_{ui} \approx p_u^\top q_i $$

* $p_u$ иногда получается интерпретировать как заинтересованность пользователя в некоторой категории товаров;
* $q_i$ иногда получается интерпретировать как принадлежность товара к определенной категории.

Будем использовать SVD:
$$ X = U \Sigma V^\top ,$$

Результат будем искать как $\hat{R}$:
 $$P = U\sqrt{\Sigma}$$
$$Q = \sqrt{\Sigma}V^\top$$
$$\hat{R} = P^\top Q$$

In [119]:
matr = train_table.fillna(0).values

u, s, vt = svds(matr, k=10)
s = np.diag(s)

p = u.dot(s ** 0.5)
q = (s ** 0.5).dot(vt)

R = p.dot(q)

In [124]:
pred = np.zeros(len(test))

for i in range(len(pred)):
    uid, sid, _ = list(test.values[i, :])

    ind1 = (train_table.index.values == uid)
    ind2 = (train_table.columns.values == sid)
    
    if len(ind1[ind1 == True]) == 0 or \
        len(ind2[ind2 == True]) == 0:
        pred[i] = 0
        continue
    
    pred[i] = R[ind1, ind2]
    
print(mean_squared_error(test.values[:, 2], pred))

9.536224899712506


### Latent Factor Model

* Будем оптимизировать следующий функционал
$$ \sum\limits_{u,i}(R_{ui} - \bar{R}_u - \bar{R}_i - \langle p_u, q_i \rangle)^2 + \lambda \sum_u\| p_u \|^2 + \mu\sum_i\| q_i \|^2 \rightarrow \min\limits_{P, Q} $$
* С помощью градиентного спуска (на каждом шаге случайно выбирая пару $(u,i)$:
$$ p_{uk} = p_{uk} + 2\alpha \left(q_{ik}(R_{ui} - \bar{R}_u - \bar{R}_i - \langle p_u, q_i \rangle) - \lambda p_{uk}\right)$$
$$ q_{ik} = q_{ik} + 2\alpha \left(p_{uk}(R_{ui} - \bar{R}_u - \bar{R}_i - \langle p_u, q_i \rangle) - \mu q_{ik}\right)$$

In [255]:
matr = train_table.fillna(0).values
num_users, num_items = matr.shape
k = 30 # количество скрытых факторов
alpha = 0.1
beta = 0.01
iterations = 6
num_samples = 5000

P = np.random.normal(scale=1./k, size=(num_users, k))
Q = np.random.normal(scale=1./k, size=(num_items, k))

b_u = np.zeros(num_users)
b_i = np.zeros(num_items)
b = matr[matr != 0].mean()

samples = [
    (i, j, matr[i, j]) for i in range(num_users) for j in range(num_items) \
        if matr[i, j] > 0
]

for i in range(iterations):
    np.random.shuffle(samples)
    
    for i, j, r in samples[:num_samples]:
        cur_pred = b + b_u[i] + b_i[j] + P[i, :].dot(Q[j, :].T)
        error = (r - cur_pred)
        
        b_u[i] += alpha * (error - beta * b_u[i])
        b_i[j] += alpha * (error - beta * b_i[j])
        
        P[i, :] += alpha * (error * Q[j, :] - beta * P[i, :])
        Q[j, :] += alpha * (error * P[i, :] - beta * Q[j, :])

In [256]:
pred = np.zeros(len(test))

for i in range(len(pred)):
    uid, sid, _ = list(test.values[i, :])

    ind1 = (train_table.index.values == uid)
    ind2 = (train_table.columns.values == sid)
    
    if len(ind1[ind1 == True]) == 0 or \
        len(ind2[ind2 == True]) == 0:
        pred[i] = 0
        continue
    
    pred[i] = b + b_u[ind1] + b_i[ind2] + P[ind1, :].dot(Q[ind2, :].T)
    
print(mean_squared_error(test.values[:, 2], pred))

1.3073876324248408


Таким образом, лучший результат показала модель Latent Factor.

Попробуем рекомендовать с помощью неё сериалы какому либо пользователю и посмотрим результаты.

In [280]:
df_shows = pd.read_csv('./serials.csv')

In [292]:
df_shows.index = df_shows.ID
df_shows.drop(columns=['ID'], inplace=True)
df_shows.head()

,Name,Rating,Viewers,Viewers %,Seasons,Date
ID,,,,,,
32756,0032,4,26.0,NaN,1.0,2007.0
25039,009-1,3,45.0,0.01,1.0,2006.0
1340,Седьмой дух,4,2704.0,0.48,1.0,2009.0
29125,07 zgłoś się,3,9.0,NaN,5.0,1976.0
2660,08th MS Team,4,61.0,0.01,1.0,2001.0


In [293]:
user_id = 1

user_ratings = df_ratings[df_ratings.user_id == user_id]

In [298]:
ind = np.argpartition(user_ratings.values[:, 2], 10)[:10]
top_show_ind = user_ratings.values[ind, 1]

In [299]:
top_show_ind

array([17448, 20391,   552,   217,  1236, 25193, 23062,   558, 43191,
       44546])

In [300]:
df_shows.loc[top_show_ind].Name

ID
17448                    Лангольеры
20391                      Призраки
552      Мастера научной фантастики
217               Воплощение страха
1236                   Святой дозор
25193                         Культ
23062                   Брайт Фоллс
558                 Вирус Андромеда
43191                 Конец детства
44546                          Мгла
Name: Name, dtype: object

In [303]:
pred = np.zeros(len(train_table.columns.values))

for i, show_id in enumerate(list(train_table.columns.values)):
    ind1 = (train_table.index.values == user_id)
    ind2 = (train_table.columns.values == show_id)
    
    if len(ind1[ind1 == True]) == 0 or \
        len(ind2[ind2 == True]) == 0:
        pred[i] = 0
        continue
    
    pred[i] = b + b_u[ind1] + b_i[ind2] + P[ind1, :].dot(Q[ind2, :].T)

In [307]:
ind = np.argpartition(pred, 15)[:15]

In [309]:
top_show_ind = train_table.columns.values[ind]

In [310]:
df_shows.loc[top_show_ind].Name

ID
55427                                  Грёзы
33972                           За пределами
51591                              Сверхлюди
37832                               Делириум
43118                           Слепое пятно
217                        Воплощение страха
32031                          Люди будущего
36409                     Герои: Возрождение
57031                                  Дождь
6208                                    Плащ
56872                           Зачарованные
44548    Мыслить как преступник: За границей
29963                            Под куполом
24474                              Революция
55120                                  Порох
Name: Name, dtype: object